Import Library

In [153]:
import os
import numpy as np
from PIL import Image
import numpy as np
import math
from numpy.random import normal

Flatten

In [154]:
# Set the input directory path
input_directory = "../Dataset/Foto_Resize_70x70"

# List all folders in the input directory
folders = [f for f in os.listdir(input_directory) if os.path.isdir(os.path.join(input_directory, f))]
print(f"Found folders: {folders}")

# Initialize arrays for inputs and outputs
X = []
Y = []

# Create a mapping from folder names to one-hot encoded labels
label_map = {folder: idx for idx, folder in enumerate(folders)}
num_classes = len(folders)

# Process each folder and photo
for folder in folders:
    folder_path = os.path.join(input_directory, folder)
    
    # Sort the list of photos to ensure consistent order
    photos = sorted([p for p in os.listdir(folder_path) if p.endswith(('.png', '.jpg', '.jpeg'))])
    
    for photo in photos:
        photo_path = os.path.join(folder_path, photo)
        image = Image.open(photo_path)
        
        # Convert the image to a numpy array and flatten it
        image_array = np.array(image).flatten()
        X.append(image_array)
        
        # Create a one-hot encoded label
        label = np.zeros(num_classes)
        label[label_map[folder]] = 1
        Y.append(label)

# Convert lists to numpy arrays
X = np.array(X)
Y = np.array(Y)

print(f"Input array shape: {X.shape}")
print(f"Output array shape: {Y.shape}")

Found folders: ['Azmira', 'David', 'Dimas', 'Fadhli', 'Fadlin', 'Hafidz', 'Haidar', 'Hanna', 'Keiko', 'Khansa', 'Mikhael', 'Puti', 'Raesa', 'Satwika', 'Toni']
Input array shape: (1384, 4900)
Output array shape: (1384, 15)


Normalize X, Print Y

In [155]:
print(X)
xmax = X.max()
xmin = X.min()
X =  (X-xmin)/(xmax-xmin)
print(X)

[[218 220 222 ... 137 112 103]
 [253 252 251 ... 137 127 116]
 [106 117 133 ... 220 219 218]
 ...
 [213 213 214 ...  55  50  48]
 [214 214 214 ...  70  68  68]
 [ 51  48  51 ... 213 213 212]]
[[0.85490196 0.8627451  0.87058824 ... 0.5372549  0.43921569 0.40392157]
 [0.99215686 0.98823529 0.98431373 ... 0.5372549  0.49803922 0.45490196]
 [0.41568627 0.45882353 0.52156863 ... 0.8627451  0.85882353 0.85490196]
 ...
 [0.83529412 0.83529412 0.83921569 ... 0.21568627 0.19607843 0.18823529]
 [0.83921569 0.83921569 0.83921569 ... 0.2745098  0.26666667 0.26666667]
 [0.2        0.18823529 0.2        ... 0.83529412 0.83529412 0.83137255]]


In [156]:
print(y)

[array([0.00285776, 0.00146788, 0.00417331, 0.00126431, 0.00706033,
       0.00388941, 0.00110341, 0.00335564, 0.00222319, 0.00341602,
       0.00236806, 0.0006355 , 0.01133345, 0.00474893, 0.010247  ]), array([0.00255077, 0.0016401 , 0.00399278, 0.00133749, 0.00745717,
       0.00843128, 0.00082738, 0.00411615, 0.00096274, 0.00220791,
       0.00290793, 0.0006524 , 0.01142937, 0.00332377, 0.00207795]), array([1.58730596e-03, 2.49436171e-03, 1.96099520e-03, 3.62429245e-03,
       6.20570670e-03, 9.40907537e-04, 5.94978807e-02, 3.13171348e-03,
       3.12165952e+19, 1.67341094e-03, 1.49722077e-03, 1.69131103e-01,
       7.93144858e-03, 1.34428846e-03, 3.50159982e+05]), array([1.60012727e-03, 2.84293280e-03, 2.10269307e-03, 3.03597684e-03,
       6.55655629e-03, 2.19451936e-03, 3.60440021e-03, 4.24223360e-03,
       8.81118334e+03, 1.26597320e-03, 2.00981593e-03, 1.46340509e-02,
       8.04397667e-03, 1.33235559e-03, 2.64242403e-01]), array([0.00271483, 0.00146749, 0.00348192, 0.00134529

Initialize Layer

In [157]:
inputLayer = 2500
hiddenLayer = 1
outputLayer = 15

Calculating Weights for Input Layer to Hidden Layer

In [158]:
#70x70
#input = 4900, hiddenLayer = 1, neuron = 2

n = 4900
stddev = np.sqrt(2/n)
hiddenNeurons = 2
v = np.random.normal(loc=0, scale = stddev, size = (n, hiddenNeurons))
print(v)
np.shape(v)

[[ 0.05272673  0.00647697]
 [-0.01979778  0.02371835]
 [-0.04096317 -0.00372062]
 ...
 [ 0.00235996 -0.00635377]
 [-0.00041907 -0.02244291]
 [-0.03023513  0.00947721]]


(4900, 2)

In [159]:
vb = np.random.normal(loc=0, scale = stddev, size = (1, hiddenNeurons))
print(vb)
np.shape(vb)

[[0.03188351 0.01867637]]


(1, 2)

Calculating Weights for Hidden Layer to Output Layer

In [160]:
#70x70
#input = 4900, hiddenLayer = 1, neuron = 2

n = 2
stddev = np.sqrt(2/n)
w = np.random.normal(loc=0, scale = stddev, size = (n, outputLayer))
print(w)
np.shape(w)

[[ 1.18005127  0.86292549  0.37574882  0.78284191  0.32699715 -0.11373246
  -1.01854664  0.99374566 -0.85455312  0.66799177 -1.34368499 -0.15161015
  -0.12864772  0.1108285  -0.15518169]
 [-1.26663532 -0.26279698  1.22369943 -1.23908871 -0.18850844  0.83533122
   0.672099    0.50044841  1.48482061 -0.21632729 -0.55442022  0.94899361
   0.3333901  -0.19465475  0.35681006]]


(2, 15)

Calculating Bias Weight from Hidden Layer to Output Layer

In [161]:
wb = np.random.normal(loc=0, scale = stddev, size = (1, outputLayer))
print(wb)
np.shape(wb)

[[ 0.47614742  0.93091763  1.54321074 -1.25418505 -0.00744106  0.533769
   0.11706276 -0.1849297  -0.52166136 -2.20503919  1.43523236  0.0162678
   2.08862288  0.7880464   0.26158642]]


(1, 15)

Feedforward

In [162]:
z_in = np.matmul(X,v) + vb
print(z_in)
np.shape(z_in)

[[-0.78877832  0.67748225]
 [-0.51705698 -0.19164256]
 [ 0.7257074  -0.03489846]
 ...
 [-0.14438883  0.5615792 ]
 [ 0.08700404 -0.1117191 ]
 [ 0.92560174 -0.36725002]]


(1384, 2)

In [163]:
# z = max([0,z_in])
z = np.maximum(0,z_in)
print(z)

[[0.         0.67748225]
 [0.         0.        ]
 [0.7257074  0.        ]
 ...
 [0.         0.5615792 ]
 [0.08700404 0.        ]
 [0.92560174 0.        ]]


In [164]:
#y_in = np.dot(z,w) + wb
y_in = np.matmul(z,w) + wb
print(y_in)

[[-0.38197553  0.75287734  2.37224539 ...  2.31448876  0.65617126
   0.50331891]
 [ 0.47614742  0.93091763  1.54321074 ...  2.08862288  0.7880464
   0.26158642]
 [ 1.33251936  1.55714904  1.81589444 ...  1.99526228  0.86847546
   0.14896992]
 ...
 [-0.23516863  0.78333631  2.2304149  ...  2.27584783  0.67873234
   0.46196353]
 [ 0.57881666  1.00599564  1.57590241 ...  2.07743001  0.79768892
   0.24808499]
 [ 1.56840493  1.72964297  1.8910045  ...  1.96954633  0.89062944
   0.11794998]]


In [165]:
# y = exponent/sumofexponent
exponents = [np.exp(i) for i in y_in]
sumofExponents = sum(exponents)
y = [(np.exp(i)/sumofExponents) for i in exponents]
print(y)
np.shape(y)

[array([7.86286310e-04, 2.01430030e-03, 4.96144518e+00, 2.87072754e-03,
       1.67821380e-03, 7.32747038e-03, 3.89991981e-03, 1.98336235e-03,
       5.05327031e-03, 6.39296675e-03, 4.28682713e-03, 4.13516311e-03,
       2.13425945e+00, 2.28577466e-03, 2.79441835e-03]), array([0.00198757, 0.00304654, 0.01179416, 0.00337581, 0.0018903 ,
       0.00200119, 0.00203936, 0.00141812, 0.00180501, 0.00648977,
       0.01597278, 0.00165317, 0.2758662 , 0.00299937, 0.00195895]), array([0.01759536, 0.02772807, 0.05114091, 0.00419914, 0.00246594,
       0.00174829, 0.00113344, 0.00341348, 0.001372  , 0.00695179,
       0.00116709, 0.00148705, 0.13432452, 0.00360593, 0.00170584]), array([0.00116645, 0.00248676, 0.10872128, 0.00307671, 0.0017844 ,
       0.0033586 , 0.00266437, 0.00163628, 0.00258035, 0.00644247,
       0.00810956, 0.0023631 , 0.67936555, 0.0026291 , 0.00228916]), array([0.00102875, 0.0023436 , 0.26172077, 0.00300976, 0.00175415,
       0.00405933, 0.0029289 , 0.00171834, 0.00299439

(1384, 15)

Back Propagation

In [ ]:
#setiap unit output
#sigma_k = (Y-y)*f'(y_in)
#gradien fungsi softmax = 

print(sigma_k)
np.shape(sigma_k)

[[9.98428046e-01 4.05740571e-06 2.46159383e+01 ... 4.55506340e+00
  5.22476579e-06 7.80877390e-06]
 [9.96028804e-01 9.28142904e-06 1.39102185e-04 ... 7.61021616e-02
  8.99622224e-06 3.83750238e-06]
 [9.65118879e-01 7.68845696e-04 2.61539240e-03 ... 1.80430757e-02
  1.30027641e-05 2.90990224e-06]
 ...
 [7.67192905e-07 4.62681408e-06 1.44479389e+00 ... 2.11506126e+00
  5.70513603e-06 9.94780254e-01]
 [5.59548641e-06 1.37852721e-05 1.89849625e-04 ... 6.35830307e-02
  9.38788053e-06 9.96153458e-01]
 [2.32641252e-03 4.58994944e-03 6.82309795e-03 ... 1.24204827e-02
  1.44679872e-05 9.96709850e-01]]


(1384, 15)